# MPI - operacje punktowe (point to point)

Stosując inferface `mpi4py` 

In [1]:
import numpy as np
import os 
print os.getenv("HOME")
wd = os.path.join( os.getenv("HOME"),"mpi_tmpdir")
if not os.path.isdir(wd):
    os.mkdir(wd)
os.chdir(wd)
print "WD is now:",os.getcwd()

/home/users/marcin.kostur
WD is now: /home/users/marcin.kostur/mpi_tmpdir


In [29]:
%%writefile mpi001.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()
print "OK, rank= ",rank,size

Writing mpi001.py


In [30]:
!mpirun -n 3 python mpi001.py



OK, rank=  2 3
OK, rank=  1 3
OK, rank=  0 3


In [31]:
!mpirun -H gpu2,gpu3  python mpi001.py



/usr/bin/ssh: /opt/puppetlabs/puppet/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
/usr/bin/ssh: /opt/puppetlabs/puppet/lib/libcrypto.so.1.0.0: no version information available (required by /usr/bin/ssh)
OK, rank=  0 2
OK, rank=  1 2


In [32]:
%%writefile mpi001.py

from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
   data = {'a': 7, 'b': 3.14}
   comm.send(data, dest=1, tag=11)
   
elif rank == 1:
   data = comm.recv(source=0, tag=11)
   print "OK, rank= ",rank,"dane: ",data

Overwriting mpi001.py


In [33]:
!mpirun -n 2 python mpi001.py

OK, rank=  1 dane:  {'a': 7, 'b': 3.14}


In [14]:
%%writefile mpi001.py
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))
if rank == 0:
   a[:] = 2
   comm.send(a[0,:], dest=1)
   
elif rank == 1:
   a[0,:] = comm.recv(source=0)
   print "OK,",np.sum(a)


Overwriting mpi001.py


In [15]:
!mpirun -n 2 python mpi001.py

OK, 4.0


In [16]:
%%writefile mpi001.py
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))
if rank == 0:
   a[:] = 2
   comm.send(a[:,0], dest=1)
   
elif rank == 1:
   a[:,0] = comm.recv(source=0)
   print "OK,",np.sum(a)

Overwriting mpi001.py


In [17]:
!mpirun -n 2 python mpi001.py

OK, 4.0


In [18]:
%%writefile mpi001.py
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))
if rank == 0:
   a[:] = 2
   comm.Send(a[0,:], dest=1)
   
elif rank == 1:
   comm.Recv(a[0,:], source=0)
   print "OK,",np.sum(a)


Overwriting mpi001.py


In [19]:
!mpirun -n 2 python mpi001.py

OK, 4.0


In [20]:
%%writefile mpi001.py
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))
if rank == 0:
   a[:] = 2
   comm.Send(a[:,0], dest=1)
   
elif rank == 1:
   comm.Recv(a[:,0], source=0)
   print "OK,",np.sum(a)


Overwriting mpi001.py


In [21]:
!mpirun -n 2 python mpi001.py

Traceback (most recent call last):
  File "mpi001.py", line 10, in <module>
    comm.Send(a[:,0], dest=1)
  File "MPI/Comm.pyx", line 272, in mpi4py.MPI.Comm.Send (src/mpi4py.MPI.c:92870)
Traceback (most recent call last):
  File "mpi001.py", line 13, in <module>
    comm.Recv(a[:,0], source=0)
  File "MPI/Comm.pyx", line 284, in mpi4py.MPI.Comm.Recv (src/mpi4py.MPI.c:93084)
  File "MPI/msgbuffer.pxi", line 406, in mpi4py.MPI.message_p2p_send (src/mpi4py.MPI.c:33688)
  File "MPI/msgbuffer.pxi", line 411, in mpi4py.MPI.message_p2p_recv (src/mpi4py.MPI.c:33765)
  File "MPI/msgbuffer.pxi", line 389, in mpi4py.MPI._p_msg_p2p.for_send (src/mpi4py.MPI.c:33550)
  File "MPI/msgbuffer.pxi", line 148, in mpi4py.MPI.message_simple (src/mpi4py.MPI.c:30349)
  File "MPI/msgbuffer.pxi", line 86, in mpi4py.MPI.message_basic (src/mpi4py.MPI.c:29359)
  File "MPI/asbuffer.pxi", line 200, in mpi4py.MPI.getbuffer (src/mpi4py.MPI.c:9310)
  File "MPI/msgbuffer.pxi", line 397, in mpi4py.MPI._p_msg_p2p.for_rec

In [22]:
%%writefile mpi001.py
import numpy as np
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

a = np.zeros((2,2))
if rank == 0:
   a[:] = 2
   buf = a[:,0].copy()
   comm.Send(buf, dest=1)
   
elif rank == 1:
   buf = np.empty(2)
   comm.Recv(buf, source=0)
   a[:,0] = buf 
   print "OK,",np.sum(a)


Overwriting mpi001.py


In [23]:
!mpirun -n 2 python mpi001.py

OK, 4.0


### execnet of NFS is not present
        
    import os
    import tempfile
    import execnet
    wd = tempfile.mkdtemp()
    os.chdir(wd)
    os.getcwd()
    rsync = execnet.RSync(wd)
    group= [execnet.makegateway('ssh=gpu2')]
    
    
    for gw in group:
        rsync.add_target(gw, wd)
        rsync.send()

